In [1]:
import os, re 
import numpy as np
import tensorflow as tf
#파일을 행단위로 끊어서 저장
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

#### 모델이 데이터를 학습할 수 있게 데이터를 전처리하는 과정을 거칩니다.

In [2]:
#문장을 사용할 수 있도록 정규화
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [3]:
corpus = []
for sentence in raw_corpus:
    if len(sentence)==0: continue
    if sentence[-1] == ":" : continue
    
    p_sentence = preprocess_sentence(sentence)
    corpus.append(p_sentence)

In [4]:
#토큰 : 텐서에 들어가있는 문자의 인덱스 , 텐서 : 단어의 인덱스로 구성
def tokenize(corpus,size = 7000):
    #7천개의 단어를 기억할 수 있는 tokenizer를 만듬
    #7천 단어에 포함되지 못한 단어는 <unk>로 변경
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = size,
        filters = ' ',
        oov_token="<unk>"
    )
    #corpus를 이용해 단어를 학습시킴
    tokenizer.fit_on_texts(corpus)
    #tokenizer를 이용해 corpus를 텐서로 변환
    tensor = tokenizer.texts_to_sequences(corpus)
    #입력 데이터의 시퀀스 길이를 일정하게 맞춰줌
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
    
    return tensor,tokenizer

In [5]:
#단어의 인덱스로 구성된 문자열로 표현된다.
tensor,tokenizer = tokenize(corpus)
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [6]:
#tokenizer에 저장된 단어
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break
# 2번 인덱스가 <start>.

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


* 텐서에 0으로 저장된 값은 문자가 비어있는것. 0은 \<pad>로 채워질 것입니다.

In [7]:
#원본 문장
print(tensor[0])
#마지막 토큰을 잘라내어 소스 문장을 생성. 마지막 문장은 pad일 가능성이 높음
src_input = tensor[:,:-1]

print(src_input[0])
#start 태그를 잘라내어 타겟 문장을 생성
tgt_input = tensor[:,1:]

print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0   0]
[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input)//BATCH_SIZE
#Tokenizer가 생성한 단어사전 내 단어 7천개와 <pad>문자 1개
VOCAB_SIZE = tokenizer.num_words + 1;

#데이터 셋을 만든다
dataset = tf.data.Dataset.from_tensor_slices((src_input,tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

#### 데이터 전처리 끝!!

#### 단어 생성기 모델 생성
###### 단어 생성기 모델의 구조
* embedding : 단어를 추상적으로 변환하는 역할을 합니다.
* RNN : 문장을 순차적으로 읽으며 단어간의 연관성을 분석합니다.
* Linear(Dense) : RNN에서 만들어낸 결과를 바탕으로 생성할 단어를 결정합니다.

In [9]:
#단어 생성기 클래스 정의
class TextGen(tf.keras.Model):
    def __init__(self,vocab_size,embedding_size,hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size,return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size,return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self,x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [10]:
embedding_size = 256
hidden_size = 1024
model = TextGen(VOCAB_SIZE,embedding_size,hidden_size)

In [11]:
for src_sample, tgt_sample in dataset.take(1): break
#배치 하나를 모델에 넣어본다.
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 1.25674662e-04,  2.26266537e-04,  2.96517654e-04, ...,
          3.63301864e-04, -4.63798679e-05, -3.29888862e-05],
        [ 2.16798071e-04,  1.65018166e-04,  4.81989700e-04, ...,
          6.58341974e-04,  6.75437987e-05,  1.66632744e-04],
        [ 1.68769766e-04,  9.84599028e-05,  6.56000338e-04, ...,
          6.67901186e-04,  4.69546540e-05, -8.93163742e-05],
        ...,
        [-6.27967878e-04,  1.34634087e-03, -2.42275349e-03, ...,
          1.42183353e-03,  1.77932973e-03, -5.98388375e-04],
        [-1.05585018e-03,  1.83710374e-03, -2.82890140e-03, ...,
          1.39157264e-03,  2.02784571e-03, -7.96724344e-04],
        [-1.43268018e-03,  2.27780477e-03, -3.19208903e-03, ...,
          1.36125286e-03,  2.23818817e-03, -9.99430777e-04]],

       [[ 1.25674662e-04,  2.26266537e-04,  2.96517654e-04, ...,
          3.63301864e-04, -4.63798679e-05, -3.29888862e-05],
        [-2.02149138e-04,  7.72706990e-05,  6.

#### 모델 shape가 (256,20,7001)로 구성되어 있는데..
* 7001은 Dense 레이어의 출력 차원 수이며 7001개의 단어의 확률 분포입니다.
* 256은 설정한 배치 사이즈입니다. 하나의 배치에 256개의 문장 데이터를 가지도록 설정한것입니다.
* 20은 문장 시퀀스의 길이입니다.

In [12]:
'''optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True,
    reduction='none'
)
#모델 학습
model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset,epochs=30)'''

"optimizer = tf.keras.optimizers.Adam()\nloss = tf.keras.losses.SparseCategoricalCrossentropy(\n    from_logits = True,\n    reduction='none'\n)\n#모델 학습\nmodel.compile(loss=loss, optimizer=optimizer)\nmodel.fit(dataset,epochs=30)"

In [13]:
def generate_text(model,tokenizer,init_sentence = "<start>",max_len = 20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input,dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    while True:
        #입력받은 문장의 텐서를 입력
        predict = model(test_tensor)
        #가장 높은 확률을 가지는 word_idx를 뽑아냄
        predict_word = tf.argmax(tf.nn.softmax(predict,axis=-1),axis=-1)[:,-1]
        #예즉된 word_index를 문장 뒤에 붙임
        test_tensor = tf.concat([test_tensor,tf.expand_dims(predict_word,axis=0)],axis=-1)
        #종료조건
        if predict_word.numpy()[0]==end_token or test_tensor.shape[1] >= max_len: 
            break
        
    
    generated = ""
    
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index]+" "
        
    return generated

#### 문장을 생성해봅시다

In [14]:
sentence = generate_text(model, tokenizer, init_sentence="<start> he")
print(sentence)
sentence = generate_text(model, tokenizer, init_sentence="<start> i")
print(sentence)

<start> he bisson bisson minion studies became bruised bruised bruised warriors shame shame shame spoons portion runn runn rod rod 
<start> i soundly making spotted spotted spotted captives captives captives captives captives sinew sinew seat mother vaward vaward vaward vaward 


## 인공지능 작사가 만들기!!

In [15]:
import glob
import os
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

In [16]:
#데이터 정제
corpos = []
for sentence in raw_corpus:
    if len(sentence)==0: continue
    if sentence[-1] == ":" : continue
    
    p_sentence = preprocess_sentence(sentence)
    corpus.append(p_sentence)

In [17]:
tensor,tokenizer = tokenize(corpus,18000)

In [18]:
#너무 긴 문장은 과도한 pad를 가지므로 잘라준다
src_input = tensor[:,:-1]
src_input = src_input[:,:15]
print(src_input[0])
tgt_input = tensor[:,1:]
tgt_input = tgt_input[:,:15]
print(tgt_input[0])

[   2  175   22 2151  320 1553    4  173   12  386   15    3    0    0
    0]
[ 175   22 2151  320 1553    4  173   12  386   15    3    0    0    0
    0]


In [19]:
from sklearn.model_selection import train_test_split

In [20]:
#테스트 데이터 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(
    src_input, 
    tgt_input, 
    test_size=0.2, 
    shuffle=True, 
    random_state=34)
enc_train.shape
#enc_val.shape

(159811, 15)

In [21]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train)//BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1;

#데이터 셋을 만든다
dataset = tf.data.Dataset.from_tensor_slices((enc_train,dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)>

In [22]:
embedding_size = 1024
hidden_size = 1500
lyricist = TextGen(VOCAB_SIZE,embedding_size,hidden_size)

In [23]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [24]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True,
    reduction='none'
)
#모델 학습
lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset,epochs=10,validation_data=(enc_val,dec_val))

Epoch 1/10
624/624 [==============================] - 311s 486ms/step - loss: 3.4640 - val_loss: 3.1349
Epoch 2/10
624/624 [==============================] - 303s 485ms/step - loss: 2.9908 - val_loss: 2.9316
Epoch 3/10
624/624 [==============================] - 303s 486ms/step - loss: 2.7766 - val_loss: 2.8060
Epoch 4/10
624/624 [==============================] - 303s 486ms/step - loss: 2.5978 - val_loss: 2.7172
Epoch 5/10
624/624 [==============================] - 304s 487ms/step - loss: 2.4327 - val_loss: 2.6502
Epoch 6/10
624/624 [==============================] - 303s 486ms/step - loss: 2.2780 - val_loss: 2.5992
Epoch 7/10
624/624 [==============================] - 304s 487ms/step - loss: 2.1307 - val_loss: 2.5594
Epoch 8/10
624/624 [==============================] - 305s 488ms/step - loss: 1.9892 - val_loss: 2.5313
Epoch 9/10
624/624 [==============================] - 304s 488ms/step - loss: 1.8542 - val_loss: 2.5163
Epoch 10/10
624/624 [==============================] - 304s 487m

In [25]:
generate_text(lyricist, tokenizer, init_sentence="<start> he", max_len=15)

'<start> he s got a mag with the <unk> <end> '

In [26]:
#10소절만 만들어보자
list = ["my","she","may","he","love","me","to","no","by","real"]
gen = []
for word in list:
    sentence = generate_text(lyricist, tokenizer, init_sentence="<start> "+word, max_len=15)
    gen.append(sentence)

gen

['<start> my name is <end> ',
 '<start> she s got me runnin round and round <end> ',
 '<start> may you arrive and it s not there <end> ',
 '<start> he s got a mag with the <unk> <end> ',
 '<start> love you , love you <end> ',
 '<start> me and my homies , so drop that <end> ',
 '<start> to the <unk> of the lamb <end> ',
 '<start> no , no , no , no , no , no , no , ',
 '<start> by the way you see me <end> ',
 '<start> real nixgaz do real things on the road to riches and diamond rings <end> ']

## 회고합시다.
* 동작 자체는 나쁘지않게 동작하는것 같은데 노래 가사라고 생각하면 봐줄만한 정도인것 같습니다.
* 다만 학습 데이터가 매우 많아 학습에 너무 시간이 오래걸려 모델의 적절한 사이즈를 찾는게 다소 어려웠습니다.  
* 모델의 검증 loss를 2.2 이하로 내려보고싶었지만 아무리 맞춰봐도 2.3이상으론 줄어들지 않아서 적당히 타협할수밖에 없었습니다.(아예 size를 대폭 올렸더니 시간만 5시간이 걸린데다가 loss도 거기서 거기..)
